In [ ]:
# !pip install --upgrade google-api-python-client

In [2]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import math 

PROJECT = 'qwiklabs-gcp-aebfb78fe0f1b1d1' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'sarah-bucket' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
os.environ['TFVERSION'] = '1.7'  # Tensorflow version

SEQ_LEN = 50

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Define the project, Bucket and Region and set them.

In [3]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

In [4]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [5]:
! pwd
! ls

/content/datalab/energy_forcasing/Sarah
ARIMA.csv		 data		  train_cloud_model.ipynb
BIG_q.ipynb		 Energy_ts	  trained
build_local_model.ipynb  hyperparam.yaml  train_local_model.ipynb
Build_model_task.ipynb	 sarah_data.csv
CNN_W2P_model		 test.json


In [6]:
CSV_COLUMNS = ['prediction_date', 'wind_speed_100m', 
  'wind_direction_100m', 
  'temperature', 
  'air_density',
  'pressure', 'precipitation', 'wind_gust', 'radiation', 
  'wind_speed', 'wind_direction', 'price']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df5 = pd.read_csv('/content/datalab/energy_forcasing/Sarah/sarah_data.csv', header = None, names = CSV_COLUMNS)
df5.describe()

,wind_speed_100m,wind_direction_100m,temperature,air_density,pressure,precipitation,wind_gust,radiation,wind_speed,wind_direction,price
count,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000
mean,4.799901,189.582975,14.920355,1.132338,935.286695,0.117364,4.872509,225.250182,3.220986,189.082629,50.153506
std,1.964154,58.126639,9.626527,0.039333,6.105083,0.245561,2.428587,266.299815,1.319704,56.712796,12.448392
min,1.372222,47.883333,-6.044444,1.036111,913.800000,0.000000,1.194444,0.000000,0.855556,50.538889,4.000000
25%,3.534722,142.598611,7.540278,1.106111,932.034722,0.000000,3.284722,0.000000,2.294444,143.750000,43.000000
50%,4.477778,191.908333,14.213889,1.131667,934.761111,0.000000,4.327778,95.000000,2.961111,191.208333,50.890000
75%,5.544444,232.043056,21.062500,1.160556,937.805556,0.100000,5.627778,416.422222,3.850000,230.865278,59.950000
max,15.044444,329.544444,41.044444,1.229444,961.238889,1.700000,16.705556,954.288889,9.738889,327.711111,85.050000


In [7]:
df2 = pd.read_csv('data/CNN_W2P_model/train-1.csv', header = None)
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.948333,0.942500,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,...,0.658333,0.500000,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667
1,0.942500,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,...,0.500000,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333
2,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,...,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500
3,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,0.834500,...,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500,1.124667
4,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,0.834500,0.816667,...,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500,1.124667,0.492167


In [8]:
df3 = pd.read_csv('data/CNN_W2P_model/valid-1.csv', header = None) #, names = CSV_COLUMNS)
df3.describe()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
count,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,...,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000
mean,0.827618,0.827851,0.827685,0.827717,0.827745,0.827840,0.827769,0.827687,0.827698,0.827717,...,0.829831,0.829818,0.829786,0.829823,0.829786,0.829683,0.829584,0.829590,0.829577,0.829446
std,0.217746,0.217454,0.217529,0.217534,0.217526,0.217463,0.217445,0.217395,0.217400,0.217403,...,0.215509,0.215509,0.215515,0.215481,0.215470,0.215397,0.215381,0.215381,0.215382,0.215333
min,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,...,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667
25%,0.707250,0.707542,0.707250,0.707250,0.707250,0.707542,0.707542,0.707542,0.707542,0.707542,...,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625
50%,0.842167,0.842833,0.842167,0.842833,0.842833,0.842833,0.842167,0.842167,0.842167,0.842833,...,0.844917,0.844833,0.844833,0.844833,0.844833,0.844833,0.844167,0.844833,0.844833,0.844167
75%,0.995667,0.995667,0.995667,0.995667,0.995667,0.995667,0.995667,0.995125,0.995125,0.995125,...,0.995000,0.995000,0.995000,0.995000,0.995000,0.994958,0.994833,0.994833,0.994833,0.994833
max,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,...,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500


This is where youre model lives and can be pushed out to the cloud.

In [9]:
! ls /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer

__init__.py  __init__.pyc  model.py  model.pyc	task.py


In [10]:
ls /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trained/cnn/export/exporter/

1537820821/


### Push out the training and evaluation data for the cloud
You can check to make sure the files were moved by looking in the "bucket" or "storage" and looking for the data files (.csv's in our case).

In [ ]:
import shutil
shutil.rmtree('/data/CNN_W2P_model ', ignore_errors=True)
os.makedirs('/data/CNN_W2P_model/')

In [ ]:
%bash
gsutil -m rm -rf gs://${BUCKET}/CNN_W2P_model/*
gsutil -m cp data/CNN_W2P_model/*.csv gs://${BUCKET}/CNN_W2P_model

### Push out to the cloud
After you run the code below - you can watch the process kick off and run on "ML Engine"

In [ ]:
%%bash
for MODEL in linear dnn cnn rnn rnn2  ; do # rnnN
  OUTDIR=gs://${BUCKET}/CNN_W2P_model/${MODEL}
  JOBNAME=CNN_W2P_model_${MODEL}$(date -u +%y%m%d_%H%M%S)
  SEQ_LEN=50
  REGION=us-central1
  gsutil -m rm -rf $OUTDIR
  echo $JOBNAME
  gcloud ml-engine jobs submit training $JOBNAME \
     --region=$REGION \
     --module-name=trainer.task \
     --package-path=${PWD}/CNN_W2P_model/trainer \
     --job-dir=$OUTDIR \
     --scale-tier=PREMIUM_1 \
     --runtime-version=$TFVERSION \
     -- \
     --train_data_path="gs://${BUCKET}/CNN_W2P_model/train-1.csv" \
     --eval_data_path="gs://${BUCKET}/CNN_W2P_model/valid-1.csv"  \
     --output_dir=$OUTDIR \
     --train_steps=2000 --sequence_length=$SEQ_LEN --model=$MODEL
done


### Tensor Boards are finky and not always willing to run.  You can try in the window - or go back to the cloud shell and run the "command line access for TB."

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/CNN_W2P_model'.format(BUCKET))

In [ ]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

In [ ]:
## command line access of the TB
print('tensorboard --logdir=gs://{}/CNN_W2P_model --port=8083'.format(BUCKET))

## Deploy the model

### Convert the data into something that can be read into the model, so strings that are 49 long where the 50th price is perdicted.

In [19]:
MODEL_NAME="CNN_W2P_model" ## you can name it w/e you want
MODEL_VERSION="ml_on_gcp" ## you can name it w/e you want
MODEL_TYPE = "linear"
MODEL_LOCATION="$(gsutil ls gs://${}/{}/{}/export/exporter/| tail -1)".format(BUCKET,MODEL_NAME,MODEL_TYPE)

In [20]:
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['MODEL_VERSION'] = MODEL_VERSION
os.environ['MODEL_TYPE'] = MODEL_TYPE
os.environ['MODEL_LOCATION'] = MODEL_LOCATION

In [21]:
# %bash
# gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}

This will delete version [ml_on_gcp]...

Do you want to continue (Y/n)?  
ERROR: (gcloud.ml-engine.versions.delete) FAILED_PRECONDITION: Field: name Error: Cannot delete the default version unless it is the only version in the model.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: Cannot delete the default version unless it is the only version in
      the model.
    field: name


In [ ]:
# %bash
# gcloud ml-engine models delete ${MODEL_NAME}

In [ ]:
%bash
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION --enable-logging

In [22]:
print(MODEL_LOCATION)

$(gsutil ls gs://$sarah-bucket/energy_price_forecast/linear/export/exporter/| tail -1)


Make data to feed to model

In [31]:
def to_csv(filename, start, N, J):
  with open(filename, 'w') as ofp:
    for ii in range(start, N):
      # nn = int(round(np.random.uniform(low = 0, high = 6466-J),0))
      seq = (df5.price[(ii):(ii+J)]/60) ## trade nn for ii to make an in seq dataset instead of random
      line = ",".join(map(str, seq))
      ofp.write(line + '\n')
      
      if len(seq) != J:
        break

import os
try:
  os.makedirs('data/CNN_W2P_model/')
except OSError:
  pass

to_csv('data/CNN_W2P_model/to_model', start = 0, N = 6596, J = 50) #  (6646 - 50) = 6596

In [33]:
model_data = pd.read_csv('/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/to_model.csv', header = None)
model_data.shape

(6597, 49)

In [ ]:
!pwd
!ls

### To get the view on the model API go to ML Engine > Models > Model Details 

In [27]:
%%writefile test.json
{"price": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}

Overwriting test.json


In [24]:
%%writefile test.json
[45.52, 45.24, 43.792, 24.0, 37.488, 31.400000000000002, 38.800000000000004, 34.792, 47.656000000000006, 32.104000000000006, 36.0, 39.88, 40.056000000000004, 39.2, 34.952, 50.42400000000001, 36.472, 42.848000000000006, 30.496, 44.712, 32.128, 52.08, 31.664, 47.88, 44.0, 51.352000000000004, 42.624, 54.879999999999995, 32.800000000000004, 31.400000000000002, 30.680000000000003, 44.608000000000004, 32.256, 50.072, 37.424, 39.88, 47.752, 44.800000000000004, 52.08, 46.952, 31.6, 24.0, 52.08, 45.760000000000005, 24.0, 40.608000000000004, 37.664, 36.080000000000005, 11.872]

Overwriting test.json


In [28]:
%bash
gcloud ml-engine predict --model CNN_W2P_model  \
                   --version ml_on_gcp \
                   --json-instances test.json

PREDICTED
[0.0]


In [29]:
from oauth2client.client import GoogleCredentials
import requests
import json

token = GoogleCredentials.get_application_default().get_access_token().access_token
#print(token)
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)

headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
{'price': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}
  ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{"predictions": [{"predicted": [0.0]}]}'


This will take a few minutes.

In [37]:
result = []
for y in range(1,model_data.shape[0]):
    raw_prices = model_data.iloc[y]
    prices = [float(price) for price in raw_prices]
    #print prices
    data = {
      'instances': [
        {
          'price': prices
        },
      ]
    }
    #print(data)
    response = requests.post(api, json=data, headers=headers)
    print(y/model_data.shape[0])
    forecast = response.json()['predictions'][0]['predicted'][0] ## (df5.price[(nn):(nn+J)])/60 ##response.content[32:-4]
    #print (forecast)
    result.append(response.content[32:-4])

0.00015158405335758679
0.00030316810671517357
0.00045475216007276033
0.0006063362134303471
0.0007579202667879339
0.0009095043201455207
0.0010610883735031075
0.0012126724268606943
0.001364256480218281
0.0015158405335758678
0.0016674245869334546
0.0018190086402910413
0.0019705926936486283
0.002122176747006215
0.002273760800363802
0.0024253448537213886
0.0025769289070789753
0.002728512960436562
0.002880097013794149
0.0030316810671517356
0.0031832651205093224
0.003334849173866909
0.003486433227224496
0.0036380172805820826
0.0037896013339396694
0.003941185387297257
0.004092769440654843
0.00424435349401243
0.0043959375473700164
0.004547521600727604
0.00469910565408519
0.004850689707442777
0.0050022737608003635
0.005153857814157951
0.005305441867515537
0.005457025920873124
0.0056086099742307105
0.005760194027588298
0.005911778080945885
0.006063362134303471
0.006214946187661058
0.006366530241018645
0.006518114294376232
0.006669698347733818
0.0068212824010914054
0.006972866454448992
0.007124450

0.06063362134303471
0.0607852053963923
0.06093678944974989
0.061088373503107474
0.06123995755646506
0.061391541609822645
0.06154312566318023
0.06169470971653782
0.06184629376989541
0.06199787782325299
0.06214946187661058
0.062301045929968164
0.06245262998332576
0.06260421403668334
0.06275579809004093
0.06290738214339851
0.0630589661967561
0.06321055025011368
0.06336213430347128
0.06351371835682885
0.06366530241018645
0.06381688646354404
0.06396847051690162
0.06412005457025921
0.06427163862361679
0.06442322267697438
0.06457480673033197
0.06472639078368955
0.06487797483704714
0.06502955889040472
0.06518114294376232
0.06533272699711991
0.06548431105047749
0.06563589510383508
0.06578747915719266
0.06593906321055025
0.06609064726390784
0.06624223131726542
0.06639381537062301
0.06654539942398059
0.06669698347733818
0.06684856753069578
0.06700015158405336
0.06715173563741095
0.06730331969076853
0.06745490374412612
0.06760648779748371
0.06775807185084129
0.06790965590419888
0.06806123995755646

0.12323783537971805
0.12338941943307564
0.12354100348643322
0.12369258753979082
0.1238441715931484
0.12399575564650599
0.12414733969986358
0.12429892375322116
0.12445050780657875
0.12460209185993633
0.12475367591329392
0.12490525996665151
0.1250568440200091
0.12520842807336668
0.12536001212672426
0.12551159618008187
0.12566318023343945
0.12581476428679703
0.1259663483401546
0.1261179323935122
0.1262695164468698
0.12642110050022737
0.12657268455358497
0.12672426860694255
0.12687585266030013
0.1270274367136577
0.12717902076701532
0.1273306048203729
0.12748218887373047
0.12763377292708808
0.12778535698044566
0.12793694103380324
0.12808852508716084
0.12824010914051842
0.128391693193876
0.12854327724723358
0.12869486130059118
0.12884644535394876
0.12899802940730634
0.12914961346066395
0.12930119751402153
0.1294527815673791
0.1296043656207367
0.1297559496740943
0.12990753372745187
0.13005911778080945
0.13021070183416705
0.13036228588752463
0.1305138699408822
0.13066545399423982
0.13081703804

0.18629680157647416
0.18644838562983174
0.18659996968318932
0.18675155373654692
0.1869031377899045
0.18705472184326208
0.18720630589661968
0.18735788994997726
0.18750947400333484
0.18766105805669245
0.18781264211005003
0.1879642261634076
0.18811581021676518
0.1882673942701228
0.18841897832348037
0.18857056237683795
0.18872214643019555
0.18887373048355313
0.1890253145369107
0.18917689859026832
0.1893284826436259
0.18948006669698347
0.18963165075034105
0.18978323480369866
0.18993481885705624
0.19008640291041382
0.19023798696377142
0.190389571017129
0.19054115507048658
0.19069273912384418
0.19084432317720176
0.19099590723055934
0.19114749128391692
0.19129907533727453
0.1914506593906321
0.19160224344398968
0.1917538274973473
0.19190541155070487
0.19205699560406245
0.19220857965742003
0.19236016371077763
0.1925117477641352
0.1926633318174928
0.1928149158708504
0.19296649992420797
0.19311808397756555
0.19326966803092316
0.19342125208428074
0.19357283613763832
0.1937244201909959
0.19387600424

0.24965893587994542
0.24981051993330303
0.2499621039866606
0.2501136880400182
0.2502652720933758
0.25041685614673337
0.25056844020009095
0.2507200242534485
0.2508716083068061
0.25102319236016374
0.2511747764135213
0.2513263604668789
0.2514779445202365
0.25162952857359405
0.25178111262695163
0.2519326966803092
0.25208428073366684
0.2522358647870244
0.252387448840382
0.2525390328937396
0.25269061694709716
0.25284220100045474
0.2529937850538123
0.25314536910716995
0.2532969531605275
0.2534485372138851
0.2536001212672427
0.25375170532060026
0.25390328937395784
0.2540548734273154
0.25420645748067305
0.25435804153403063
0.2545096255873882
0.2546612096407458
0.25481279369410337
0.25496437774746095
0.2551159618008186
0.25526754585417616
0.25541912990753374
0.2555707139608913
0.2557222980142489
0.2558738820676065
0.25602546612096405
0.2561770501743217
0.25632863422767926
0.25648021828103684
0.2566318023343944
0.256783386387752
0.2569349704411096
0.25708655449446716
0.2572381385478248
0.25738972

...

0.7057753524329241
0.7059269364862817
0.7060785205396393
0.7062301045929968
0.7063816886463544
0.706533272699712
0.7066848567530696
0.7068364408064272
0.7069880248597847
0.7071396089131423
0.7072911929664999
0.7074427770198575
0.707594361073215
0.7077459451265727
0.7078975291799303
0.7080491132332879
0.7082006972866455
0.708352281340003
0.7085038653933606
0.7086554494467182
0.7088070335000758
0.7089586175534334
0.709110201606791
0.7092617856601485
0.7094133697135061
0.7095649537668637
0.7097165378202213
0.709868121873579
0.7100197059269365
0.7101712899802941
0.7103228740336517
0.7104744580870093
0.7106260421403668
0.7107776261937244
0.710929210247082
0.7110807943004396
0.7112323783537972
0.7113839624071547
0.7115355464605123
0.7116871305138699
0.7118387145672276
0.7119902986205852
0.7121418826739427
0.7122934667273003
0.7124450507806579
0.7125966348340155
0.712748218887373
0.7128998029407306
0.7130513869940882
0.7132029710474458
0.7133545551008034
0.713506139154161
0.7136577232075185
0

0.7718659996968319
0.7720175837501895
0.772169167803547
0.7723207518569046
0.7724723359102622
0.7726239199636198
0.7727755040169774
0.772927088070335
0.7730786721236926
0.7732302561770502
0.7733818402304078
0.7735334242837654
0.773685008337123
0.7738365923904805
0.7739881764438381
0.7741397604971957
0.7742913445505533
0.7744429286039108
0.7745945126572684
0.774746096710626
0.7748976807639836
0.7750492648173413
0.7752008488706988
0.7753524329240564
0.775504016977414
0.7756556010307716
0.7758071850841292
0.7759587691374867
0.7761103531908443
0.7762619372442019
0.7764135212975595
0.776565105350917
0.7767166894042746
0.7768682734576322
0.7770198575109899
0.7771714415643475
0.777323025617705
0.7774746096710626
0.7776261937244202
0.7777777777777778
0.7779293618311354
0.778080945884493
0.7782325299378505
0.7783841139912081
0.7785356980445657
0.7786872820979233
0.7788388661512808
0.7789904502046384
0.7791420342579961
0.7792936183113537
0.7794452023647113
0.7795967864180688
0.7797483704714264
0

0.838259815067455
0.8384113991208125
0.8385629831741701
0.8387145672275277
0.8388661512808853
0.8390177353342428
0.8391693193876004
0.839320903440958
0.8394724874943156
0.8396240715476732
0.8397756556010307
0.8399272396543883
0.8400788237077459
0.8402304077611036
0.8403819918144612
0.8405335758678187
0.8406851599211763
0.8408367439745339
0.8409883280278915
0.841139912081249
0.8412914961346066
0.8414430801879642
0.8415946642413218
0.8417462482946794
0.841897832348037
0.8420494164013945
0.8422010004547522
0.8423525845081098
0.8425041685614674
0.842655752614825
0.8428073366681825
0.8429589207215401
0.8431105047748977
0.8432620888282553
0.8434136728816128
0.8435652569349704
0.843716840988328
0.8438684250416856
0.8440200090950432
0.8441715931484007
0.8443231772017584
0.844474761255116
0.8446263453084736
0.8447779293618312
0.8449295134151887
0.8450810974685463
0.8452326815219039
0.8453842655752615
0.845535849628619
0.8456874336819766
0.8458390177353342
0.8459906017886918
0.8461421858420494
0

0.9041988782780052
0.9043504623313627
0.9045020463847203
0.9046536304380779
0.9048052144914355
0.904956798544793
0.9051083825981506
0.9052599666515082
0.9054115507048659
0.9055631347582235
0.905714718811581
0.9058663028649386
0.9060178869182962
0.9061694709716538
0.9063210550250114
0.906472639078369
0.9066242231317265
0.9067758071850841
0.9069273912384417
0.9070789752917993
0.9072305593451568
0.9073821433985145
0.9075337274518721
0.9076853115052297
0.9078368955585873
0.9079884796119448
0.9081400636653024
0.90829164771866
0.9084432317720176
0.9085948158253752
0.9087463998787327
0.9088979839320903
0.9090495679854479
0.9092011520388055
0.909352736092163
0.9095043201455207
0.9096559041988783
0.9098074882522359
0.9099590723055935
0.910110656358951
0.9102622404123086
0.9104138244656662
0.9105654085190238
0.9107169925723814
0.910868576625739
0.9110201606790965
0.9111717447324541
0.9113233287858117
0.9114749128391694
0.911626496892527
0.9117780809458845
0.9119296649992421
0.9120812490525997
0.

0.9704411095952705
0.9705926936486282
0.9707442777019858
0.9708958617553434
0.971047445808701
0.9711990298620585
0.9713506139154161
0.9715021979687737
0.9716537820221313
0.9718053660754888
0.9719569501288464
0.972108534182204
0.9722601182355616
0.9724117022889192
0.9725632863422768
0.9727148703956344
0.972866454448992
0.9730180385023496
0.9731696225557072
0.9733212066090647
0.9734727906624223
0.9736243747157799
0.9737759587691375
0.973927542822495
0.9740791268758526
0.9742307109292102
0.9743822949825678
0.9745338790359254
0.974685463089283
0.9748370471426406
0.9749886311959982
0.9751402152493558
0.9752917993027134
0.975443383356071
0.9755949674094285
0.9757465514627861
0.9758981355161437
0.9760497195695013
0.9762013036228588
0.9763528876762164
0.976504471729574
0.9766560557829317
0.9768076398362893
0.9769592238896468
0.9771108079430044
0.977262391996362
0.9774139760497196
0.9775655601030772
0.9777171441564347
0.9778687282097923
0.9780203122631499
0.9781718963165075
0.978323480369865
0.